In [166]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [167]:
# Загрузка данных
x_train = np.load('datasets/dataset2/x_train.npy')
y_train = np.load('datasets/dataset2/y_train.npy')

x_test = np.load('datasets/dataset2/x_test.npy')
y_test = np.load('datasets/dataset2/y_test.npy')

print(x_train[:5])
print(y_train)

[[ 1.97258568  0.33000871]
 [ 1.97754947  0.29882787]
 [ 0.66403114 -0.74770492]
 [ 1.87515029  0.69556553]
 [ 1.18067338  1.61431421]]
[1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 

In [168]:
class SelfmadeLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, reg_strength=0.01):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.reg_strength = reg_strength
        self.models = {}  # Словарь моделей для каждого класса

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def log_loss(self, y, predictions):
        epsilon = 1e-15
        predictions = np.clip(predictions, epsilon, 1 - epsilon)
        loss = - (y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
        return np.mean(loss)

    def fit(self, X, y):
        unique_classes = np.unique(y)

        for cls in unique_classes:
            binary_y = (y == cls).astype(int)  # Преобразуем к бинарной классификации
            model = self.train_binary_model(X, binary_y)
            self.models[cls] = model

    def train_binary_model(self, X, y):
        n_samples, n_features = X.shape
        weights = np.zeros(n_features)
        bias = 0

        for _ in range(self.n_iterations):
            model = np.dot(X, weights) + bias
            predictions = self.sigmoid(model)

            dw = (1/n_samples) * (np.dot(X.T, (predictions - y)) + self.reg_strength * weights)
            db = (1/n_samples) * np.sum(predictions - y)

            weights -= self.learning_rate * dw
            bias -= self.learning_rate * db

        return weights, bias

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models)))

        for cls, model in self.models.items():
            weights, bias = model
            model_predictions = np.dot(X, weights) + bias
            predictions[:, cls] = self.sigmoid(model_predictions)

        # Выбираем класс с наибольшей вероятностью
        return np.argmax(predictions, axis=1)

In [169]:
# Инициализация модели
model = LogisticRegression(
    solver='liblinear',
    C=100.0,
    random_state=0
)

# Обучение модели
model.fit(x_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Точность модели: {accuracy * 100:.2f}%')
print(y_pred)

Точность модели: 66.00%
[0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 1.]


In [170]:
# Используем класс
selfmade_model = SelfmadeLogisticRegression()
selfmade_model.fit(x_train, y_train)
selfmade_predictions = selfmade_model.predict(x_test)

print(selfmade_predictions)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# Сравнение результатов
selfmade_accuracy = accuracy_score(y_test, selfmade_predictions)

In [ ]:
print(f'Точность самописной модели: {selfmade_accuracy * 100:.2f}%')